# LAB 3: How to setup a project from Scratch

In [1]:
!rm -rf speech-emotion-recognition-25

# Step 1: Clone your project from Github

In [2]:
#main
#!git clone https://github.com/MatteoPaglia/speech-emotion-recognition-25.git

#             nome branch

!git clone -b RavdnessClass https://github.com/MatteoPaglia/speech-emotion-recognition-25.git

Cloning into 'speech-emotion-recognition-25'...
remote: Enumerating objects: 270, done.
remote: Counting objects: 100% (176/176), done.
remote: Compressing objects: 100% (117/117), done.
remote: Total 270 (delta 124), reused 103 (delta 55), pack-reused 94 (from 3)
Receiving objects: 100% (270/270), 102.72 KiB | 1.63 MiB/s, done.
Resolving deltas: 100% (160/160), done.


In [3]:
!ls

sample_data  speech-emotion-recognition-25


In [4]:
# %cd mldl_project_skeleton
%cd speech-emotion-recognition-25

/content/speech-emotion-recognition-25


In [5]:
!ls

checkpoints  dataset  models	 requirements.txt		  train.py
data	     eval.py  README.md  SpeechEmotionRecnognition.ipynb  utils


# Step 2: Packages Installation


In [6]:
%pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 36.2 MB/s eta 0:00:00a 0:00:01


In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

NameError: name 'torch' is not defined

# Step 3: Dataset Setup
## Different options
- First one is downloading using a script that places the data in the download folder (usually recommended)
- Second one is uploading the dataset to your personal/institutional Google Drive and load it from there ([Read More](https://saturncloud.io/blog/google-colab-how-to-read-data-from-my-google-drive/))
- Place the download script directly here on colab

You are free to do as you please in this phase.


In [7]:
# ============================================================================
# SMART DATASET DOWNLOAD - Scarica solo se non presente
# ============================================================================

from pathlib import Path

ravdess_path = Path('./ravdess')
iemocap_path = Path('./iemocap')

# Verifica se i dataset esistono già
ravdess_exists = ravdess_path.exists() and any(ravdess_path.iterdir())
iemocap_exists = iemocap_path.exists() and any(iemocap_path.iterdir())

if ravdess_exists and iemocap_exists:
    print("✅ Dataset già presenti, skip download")
    print(f"   📁 RAVDESS: {ravdess_path}")
    print(f"   📁 IEMOCAP: {iemocap_path}")
else:
    print("⬇️  Download dataset in corso...")
    !python utils/download_dataset.py

In [8]:
# ============================================================================
# ESPLORAZIONE STRUTTURA DATASET
# ============================================================================

from pathlib import Path
from collections import defaultdict

def analyze_dataset_structure(dataset_path, dataset_name):
    """Analizza e mostra la struttura di un dataset"""
    print("=" * 80)
    print(f"📁 STRUTTURA DATASET: {dataset_name.upper()}")
    print("=" * 80)
    
    path = Path(dataset_path)
    
    if not path.exists():
        print(f"❌ Directory non trovata: {dataset_path}")
        return
    
    # 1. STATISTICHE GENERALI
    print("\n📊 STATISTICHE GENERALI:")
    print("-" * 80)
    
    total_files = 0
    total_size = 0
    file_extensions = defaultdict(int)
    
    for item in path.rglob('*'):
        if item.is_file():
            total_files += 1
            total_size += item.stat().st_size
            ext = item.suffix.lower() or '[no extension]'
            file_extensions[ext] += 1
    
    print(f"  • Totale file: {total_files}")
    print(f"  • Dimensione totale: {total_size / (1024**3):.2f} GB")
    print(f"  • Tipi di file:")
    for ext, count in sorted(file_extensions.items(), key=lambda x: x[1], reverse=True):
        print(f"      {ext}: {count} file(s)")
    
    # 2. STRUTTURA DIRECTORY (primi 3 livelli)
    print("\n📂 STRUTTURA DIRECTORY (primi 3 livelli):")
    print("-" * 80)
    
    def print_tree(directory, prefix="", max_depth=3, current_depth=0):
        if current_depth >= max_depth:
            return
        
        try:
            items = sorted(directory.iterdir(), key=lambda x: (not x.is_dir(), x.name))
            dirs = [item for item in items if item.is_dir()]
            files = [item for item in items if item.is_file()]
            
            # Mostra prime 3 directory
            for i, item in enumerate(dirs[:3]):
                is_last = (i == len(dirs[:3]) - 1) and len(files) == 0
                print(f"{prefix}{'└── ' if is_last else '├── '}📁 {item.name}/")
                
                extension = "    " if is_last else "│   "
                print_tree(item, prefix + extension, max_depth, current_depth + 1)
            
            if len(dirs) > 3:
                print(f"{prefix}└── ... altre {len(dirs) - 3} cartelle")
            
            # Mostra primi 3 file
            if files and current_depth < max_depth - 1:
                for i, item in enumerate(files[:3]):
                    is_last = i == len(files[:3]) - 1
                    size_mb = item.stat().st_size / (1024**2)
                    print(f"{prefix}{'└── ' if is_last else '├── '}📄 {item.name} ({size_mb:.2f} MB)")
                
                if len(files) > 3:
                    print(f"{prefix}└── ... altri {len(files) - 3} file")
        
        except PermissionError:
            print(f"{prefix}❌ Accesso negato")
    
    print_tree(path)
    
    # 3. ESEMPIO FILE PATHS (primi 5)
    print("\n📝 ESEMPIO PERCORSI FILE (primi 5):")
    print("-" * 80)
    
    all_files = list(path.rglob('*'))
    file_list = [f for f in all_files if f.is_file()][:5]
    
    for f in file_list:
        relative_path = f.relative_to(path)
        size_mb = f.stat().st_size / (1024**2)
        print(f"  • {relative_path}")
        print(f"    Dimensione: {size_mb:.2f} MB")
    
    # 4. DISTRIBUZIONE PER LIVELLO
    print("\n📊 DISTRIBUZIONE FILE PER LIVELLO:")
    print("-" * 80)
    
    level_distribution = defaultdict(int)
    for item in path.rglob('*'):
        if item.is_file():
            depth = len(item.relative_to(path).parts) - 1
            level_distribution[depth] += 1
    
    for level in sorted(level_distribution.keys()):
        print(f"  Livello {level}: {level_distribution[level]} file(s)")
    
    print("\n" + "=" * 80 + "\n")


# ============================================================================
# ANALISI RAVDESS
# ============================================================================

print("\n" + "🎵" * 40)
print("ANALISI DATASET RAVDESS")
print("🎵" * 40 + "\n")

analyze_dataset_structure("./ravdess", "RAVDESS")


# ============================================================================
# ANALISI IEMOCAP
# ============================================================================

print("\n" + "🎙️" * 40)
print("ANALISI DATASET IEMOCAP")
print("🎙️" * 40 + "\n")

analyze_dataset_structure("./iemocap", "IEMOCAP")

# Step 3.5: Improvised Vocal Recording

Model IEMOCAP dataset to extract only improvised audio. 

In [ ]:
## insert code here 

In [ ]:
# Create Data Loaders

# Step 3.6: Test Ravdness 

Test Ravdness dataset

In [9]:
# ============================================================================
# TEST CUSTOM RAVDESS DATASET - TRAIN/VALIDATION/TEST SPLIT
# ============================================================================

from dataset.custom_ravdess_dataset import CustomRAVDESSDataset
from collections import Counter

print("=" * 80)
print("🧪 TEST DATASET RAVDESS - TRAIN/VALIDATION/TEST SPLIT")
print("=" * 80)

# 1. Inizializza i tre dataset (SENZA validation_ratio, test_ratio, seed)
print("\n📦 Inizializzazione datasets...")
print("-" * 80)

train_dataset = CustomRAVDESSDataset(
    dataset_root='./ravdess',
    split='train'
)

print("\n" + "-" * 80)

val_dataset = CustomRAVDESSDataset(
    dataset_root='./ravdess',
    split='validation'
)

print("\n" + "-" * 80)

test_dataset = CustomRAVDESSDataset(
    dataset_root='./ravdess',
    split='test'
)


# ============================================================================
# STATISTICHE GENERALI
# ============================================================================

print("\n" + "=" * 80)
print("📊 STATISTICHE DATASET")
print("=" * 80)

total_samples = len(train_dataset) + len(val_dataset) + len(test_dataset)

print(f"\n📈 Numero di samples:")
print(f"   Train:      {len(train_dataset):4d} samples ({len(train_dataset)/total_samples*100:5.1f}%)")
print(f"   Validation: {len(val_dataset):4d} samples ({len(val_dataset)/total_samples*100:5.1f}%)")
print(f"   Test:       {len(test_dataset):4d} samples ({len(test_dataset)/total_samples*100:5.1f}%)")
print(f"   {'─'*50}")
print(f"   Totale:     {total_samples:4d} samples")

# ============================================================================
# DISTRIBUZIONE EMOZIONI
# ============================================================================

print("\n" + "=" * 80)
print("🎭 DISTRIBUZIONE EMOZIONI")
print("=" * 80)

for name, dataset in [('TRAIN', train_dataset), ('VALIDATION', val_dataset), ('TEST', test_dataset)]:
    print(f"\n{name}:")
    emotions = [s['metadata']['emotion_label'] for s in dataset.samples]
    distribution = Counter(emotions)
    
    for emotion in sorted(CustomRAVDESSDataset.EMOTION_DICT.values()):
        count = distribution.get(emotion, 0)
        percentage = (count / len(dataset)) * 100 if len(dataset) > 0 else 0
        bar = '█' * int(percentage / 5)
        print(f"   {emotion:10s}: {count:3d} samples ({percentage:5.1f}%) {bar}")

# ============================================================================
# VERIFICA SPEAKER-INDEPENDENT SPLIT
# ============================================================================

print("\n" + "=" * 80)
print("🔍 VERIFICA SPEAKER-INDEPENDENT SPLIT")
print("=" * 80)

train_actors = set([int(s['metadata']['actor']) for s in train_dataset.samples])
val_actors = set([int(s['metadata']['actor']) for s in val_dataset.samples])
test_actors = set([int(s['metadata']['actor']) for s in test_dataset.samples])

print(f"\n👥 Attori per set:")
print(f"   Train ({len(train_actors)} attori):      {sorted(train_actors)}")
print(f"   Validation ({len(val_actors)} attori):   {sorted(val_actors)}")
print(f"   Test ({len(test_actors)} attori):        {sorted(test_actors)}")

# Verifica overlap
overlap_train_val = train_actors & val_actors
overlap_train_test = train_actors & test_actors
overlap_val_test = val_actors & test_actors

print(f"\n🔎 Verifica overlap attori:")
no_overlap = not overlap_train_val and not overlap_train_test and not overlap_val_test

if no_overlap:
    print("   ✅ NESSUN OVERLAP! Split speaker-independent corretto!")
else:
    print("   ❌ ATTENZIONE: Ci sono overlap!")
    if overlap_train_val:
        print(f"      Train-Validation overlap: {overlap_train_val}")
    if overlap_train_test:
        print(f"      Train-Test overlap: {overlap_train_test}")
    if overlap_val_test:
        print(f"      Validation-Test overlap: {overlap_val_test}")

# ============================================================================
# VERIFICA BILANCIAMENTO GENERE
# ============================================================================

print("\n" + "=" * 80)
print("👨👩 VERIFICA BILANCIAMENTO GENERE")
print("=" * 80)

for name, actors in [('Train', train_actors), ('Validation', val_actors), ('Test', test_actors)]:
    males = [a for a in actors if a % 2 == 1]  # Dispari = maschi
    females = [a for a in actors if a % 2 == 0]  # Pari = femmine
    
    print(f"\n{name}:")
    print(f"   👨 Maschi:  {len(males)} attori → {sorted(males)}")
    print(f"   👩 Femmine: {len(females)} attori → {sorted(females)}")
    print(f"   Ratio: {len(males)}M / {len(females)}F")

# ============================================================================
# VERIFICA FILTRI SU CAMPIONI CASUALI
# ============================================================================

print("\n" + "=" * 80)
print("🔍 VERIFICA FILTRI SU CAMPIONI CASUALI")
print("=" * 80)

import random
random.seed(42)

# Prendi 5 samples casuali dal training set
sample_indices = random.sample(range(len(train_dataset)), min(5, len(train_dataset)))

print("\n📋 Verifica su 5 samples casuali dal training set:")
all_filters_ok = True

for i, idx in enumerate(sample_indices, 1):
    sample = train_dataset.samples[idx]
    metadata = sample['metadata']
    
    # Verifica filtri
    modality_ok = metadata['modality'] == '03'
    vocal_ok = metadata['vocal_channel'] == '01'
    emotion_ok = metadata['emotion'] in ['01', '03', '04', '05']
    
    status = "✅" if (modality_ok and vocal_ok and emotion_ok) else "❌"
    
    print(f"\n  {status} Sample {i} (index {idx}):")
    print(f"     📄 File: {sample['path'].name}")
    print(f"     🎬 Modality: {metadata['modality']} {'✓' if modality_ok else '✗ (should be 03)'}")
    print(f"     🎤 Vocal: {metadata['vocal_channel']} {'✓' if vocal_ok else '✗ (should be 01)'}")
    print(f"     😊 Emotion: {metadata['emotion']} ({metadata['emotion_label']}) {'✓' if emotion_ok else '✗'}")
    print(f"     👤 Actor: {metadata['actor']} ({'M' if int(metadata['actor']) % 2 == 1 else 'F'})")
    print(f"     💪 Intensity: {metadata['intensity']}")
    
    if not (modality_ok and vocal_ok and emotion_ok):
        all_filters_ok = False

# ============================================================================
# RIEPILOGO FINALE
# ============================================================================

print("\n" + "=" * 80)
print("✅ RIEPILOGO FINALE")
print("=" * 80)

# Verifica split fisso
expected_train = set(range(1, 21))  # 1-20
expected_val = {21, 22}
expected_test = {23, 24}

checks = []
checks.append(("Nessun overlap attori", no_overlap))
checks.append(("Tutti i filtri corretti", all_filters_ok))
checks.append(("4 emozioni presenti", len(train_dataset) > 0 and len(val_dataset) > 0 and len(test_dataset) > 0))
checks.append(("Train: attori 1-20", train_actors == expected_train))
checks.append(("Validation: attori 21-22", val_actors == expected_val))
checks.append(("Test: attori 23-24", test_actors == expected_test))

print()
for check_name, check_result in checks:
    status = "✅" if check_result else "❌"
    print(f"   {status} {check_name}")

all_checks = all(result for _, result in checks)

print("\n" + "=" * 80)
if all_checks:
    print("🎉 TUTTI I TEST SUPERATI! Dataset pronto per il training!")
    print(f"📊 Split: {len(train_dataset)} train, {len(val_dataset)} val, {len(test_dataset)} test")
else:
    print("⚠️  Alcuni test non sono passati. Controlla i messaggi sopra.")
print("=" * 80)

# Step 3.7: Create DataLoader

Create DataLoader for both Datasets

In [ ]:
from dataset.custom_ravdess_dataset import CustomRAVDESSDataset

train_dataset = CustomRAVDESSDataset(dataset_root='./ravdess', split='train')
val_dataset = CustomRAVDESSDataset(dataset_root='./ravdess', split='validation')
test_dataset = CustomRAVDESSDataset(dataset_root='./ravdess', split='test')


from utils.create_dataloaders import create_dataloaders

train_loader, val_loader, test_loader = create_dataloaders(
    train_dataset, val_dataset, test_dataset,
    batch_size=32, device=device, dataset_name='RAVDESS'
)

# Step 4: Train your model and visualize training

In [ ]:
#%env WANDB_API_KEY="7ade30086de7899bed412e3eb5c2da065c146f90"

In [ ]:
#!python train.py

# Step 5: Evaluate your model

1.   List item
2.   List item



In [ ]:
#!python eval.py